# Applied `pandas` examples: reading, joining, filtering, aggregating, and exporting

This cell was changed from `code` to `markdown` at the top of the page. It lets me write nicely-formatted text that does not get executed like code.

[Markdown is a stripped-down method for making simple text look nice. This is a link that will take you to GitHub's guide on the topic](https://guides.github.com/features/mastering-markdown/)

A few notes about file/folder paths in Python:
- Mac OS uses forward slashes, like `"/path/to/somewhere"`
- Windows uses back slashes and a drive name, like `"drive:\path\to\somewhere"`
- Since backslashes have a special meaning in Python you have two choices on Windows:
    - Double up your slashes, like `"drive:\\path\\to\\somewhere"`
    - Or, put the letter `r` in front of the string, like `r"drive:\path\to\somewhere"`
    - Using `r` makes life easier than doubling up the slashes as you can copy/paste folder and filepaths and use them as-is

In [1]:
import pandas as pd

data_folder_on_macos = "/Volumes/GoogleDrive/Shared drives/Long Range Plan/Models/UrbanSim/Model Development/Base Year Migration/FinalAllocation20201006/fromSol"
data_folder_on_windows = r"G:\Shared drives\Long Range Plan\Models\UrbanSim\Model Development\Base Year Migration\FinalAllocation20201006\fromSol"

### I am going to use `data_folder_on_macos`, but when you run this on your machine you should swap in `data_folder_on_windows`

Just remove the pound sign and single space in front of the second line in the cell below.

In [2]:
data_folder = data_folder_on_macos
# data_folder = data_folder_on_windows

### There are a variety of ways to list the contents of a folder.

`os` and `pathlib` are two different built-in Python modules we can use. For simplicity I'll use `os`, which does "operating system" stuff. Also, note that I'm importing it a few cells deep into the notebook. This works fine, but is a violation of a best-practice, which is to do all of your imports up front in the first set of lines in your script. 

In [3]:
import os

csvs_in_folder = os.listdir(data_folder)

In [4]:
# You can also leave notes with the standard "comment" style in python, by prefixing a line with a pound sign

# Once you've created a variable, you can execute a cell with that variable to inspect it.
# Let's see all of the filenames in the folder:

csvs_in_folder

['allocated_hh_34_005_2017_adj.csv',
 'allocated_p_34_005_2017_adj.csv',
 'allocated_hh_34_007_2017_adj.csv',
 'allocated_p_34_007_2017_adj.csv',
 'allocated_hh_34_015_2017_adj.csv',
 'allocated_p_34_015_2017_adj.csv',
 'allocated_hh_34_021_2017_adj.csv',
 'allocated_p_34_021_2017_adj.csv',
 'allocated_hh_42_017_2017_adj.csv',
 'allocated_p_42_017_2017_adj.csv',
 'allocated_hh_42_029_2017_adj.csv',
 'allocated_p_42_029_2017_adj.csv',
 'allocated_hh_42_045_2017_adj.csv',
 'allocated_p_42_045_2017_adj.csv',
 'allocated_hh_42_091_2017_adj.csv',
 'allocated_p_42_091_2017_adj.csv',
 'allocated_hh_42_101_2017_adj.csv',
 'allocated_p_42_101_2017_adj.csv',
 'adjusted_units_20201006.csv',
 'aggregated outputs']

In [5]:
# Let's loop through the list of files and place them into an appropriate list, either for households or people

household_files = []
person_files = []

for filename in csvs_in_folder:
    if "_hh_" in filename:
        household_files.append(filename)
    elif "_p_" in filename:
        person_files.append(filename)

In [6]:
# Now let's read each household CSV into a dataframe and drop it into a list of all household dataframes

hh_dataframes = []

for hh_file in household_files:

    # Build the full path to the file by joining the folder name to the filename
    filepath = os.path.join(data_folder, hh_file)
    
    df = pd.read_csv(filepath)
    
    hh_dataframes.append(df)

### You can merge a list of dataframes together using `pd.concat(my_list)`

Make sure to set it equal to something so that you can use it later. I.E. `new_df = pd.concat([df1, df2, df3])`

In [7]:
hh_data = pd.concat(hh_dataframes)

### To confirm it worked, we can check how many unique county IDs are in our new dataframe
The syntax here is `dataframe["column_name"].unique()`

In [8]:
hh_data["county"].unique()

array([  5,   7,  15,  21,  17,  29,  45,  91, 101])

In [9]:
# Now let's repeat the process for the person tables

person_dataframes = []

for p_file in person_files:

    filepath = os.path.join(data_folder, p_file)
    df = pd.read_csv(filepath)
    
    person_dataframes.append(df)
    
person_data = pd.concat(person_dataframes)

### As an aside, you can define functions with inputs

This is advisable anytime you have a chunk of code that you've repeated. This is known as "DRY", or "Don't Repeat Yourself".

Functions start with `def` and usually `return` something. Here's how I could have done this:

In [10]:
# This is the function definition

def merge_list_of_csvs(folder, list_of_filenames):
    all_dataframes = []
    
    for single_file in list_of_filenames:
        filepath = os.path.join(folder, single_file)
        df = pd.read_csv(filepath)
        all_dataframes.append(df)
        
    merged_df = pd.concat(all_dataframes)
    return merged_df


# Now I'm using the function twice, once for person files and once for household files
person_data = merge_list_of_csvs(data_folder, person_files)
hh_data = merge_list_of_csvs(data_folder, household_files)

### If you're curious about how many rows/columns are in a dataframe:

Use `df.shape` to output `(row count, column count)`

In [11]:
# In this case, we have over 5.5 million rows and 14 columns. Definitely too big for Excel!

person_data.shape

(5547041, 14)

# Joining dataframes together

Both household and person tables share a `"household_id"` column that we'll use to join them.

The result will be a new dataframe that has a row for each person and the household's info appended in additional columns.

`Pandas` is kind of tricky from a language point of view. You'd think `pd.join()` would be the right function here, but we actually want to use `pd.merge()`

In [12]:
combined_df = pd.merge(person_data, hh_data, on="household_id")

#### Let's confirm the table size so we can be sure it worked properly

`combined_df` should have the same number of rows as the starting `person_data` table.

`df.shape[0]` lets you grab the row count. I'll write a formula that will tell us if they match using `True` or `False`.

The double equal sign `==` lets us test if two things are the same.

This is a nice way to build QAQC directly into your script and ensure that your output matches what you'd expect

In [13]:
combined_df.shape[0] == person_data.shape[0]

True

## Now let's do a basic aggregation

We'll start by getting the total number of people by block group

In [14]:
# The code below groups on the "block_id" column and the .size() bit counts the number of rows in each group

combined_df.groupby(["block_id"]).size()

block_id
340057001021001    47
340057001021002    23
340057001021003    40
340057001021004    46
340057001021005    34
                   ..
421019809001173    12
421019809001203     9
421019809001204     1
421019809001205     7
421019809001206     8
Length: 72643, dtype: int64



The output of `df.group_by()` is a `pandas.Series`, which is equivalent to a column. 

We'll drop this series into a new dataframe and then rename the 'count' column, which by default will be named `0`

In [15]:
# In order to save this to a new dataframe, we'll set it equal to a new variable and drop it into pd.DataFrame()

block_id_series = combined_df.groupby(["block_id"]).size()

block_id_df = pd.DataFrame(block_id_series)

In [16]:
# Rename column "0" as "total_people". The "inplace=True" argument modifies the dataframe and means we don't need to assign it to a new variable

rename_dictionary = {0: "total_people"}

block_id_df.rename(columns=rename_dictionary, inplace=True)

In [17]:
block_id_df

,total_people
block_id,
340057001021001,47
340057001021002,23
340057001021003,40
340057001021004,46
340057001021005,34
...,...
421019809001173,12
421019809001203,9
421019809001204,1


# Now let's use some arbitrary age filters and then re-aggregate

For this first example, we'll do everyone over the age of 50.

The syntax is kind of weird here, but basically we get a true/false for each row if it meets the condition, and then all false rows are dropped

In [18]:
over_50_df = combined_df[combined_df["age"] >= 50]

In [19]:
print("Min age:", over_50_df["age"].min())
print("Max age:", over_50_df["age"].max())

Min age: 50
Max age: 95


Chaining multiple filters together is possible, but a bit more verbose.

Note that we wrap each condition inside parentheses and use the `&` symbol to say that it must meet both conditions.

This example filters to all people between 30 and 50

In [20]:
df_30_to_50 = combined_df[(combined_df["age"] >= 30) & (combined_df["age"] < 50)]

In [21]:
print("Min age:", df_30_to_50["age"].min())
print("Max age:", df_30_to_50["age"].max())

Min age: 30
Max age: 49


We now have two new dataframes, `over_50_df` and `df_30_to_50`.

Each of these can be aggregated as before. I'll move forward with `df_30_to_50`

In [22]:
age_filtered_series = df_30_to_50.groupby(["block_id"]).size()

age_filtered_df = pd.DataFrame(age_filtered_series)

rename_dictionary = {0: "people_30_thru_49"}

age_filtered_df.rename(columns=rename_dictionary, inplace=True)

# We have what we need, now we want to get out of Python!

All dataframes have the ability to be written out to file, but be aware of your row/column count. Excel maxes out at 1.04 million rows 16k columns.

You're more likely to hit the row limit than the column limit. Either way, if you're close to the limit you'll definitely benefit from using SQL to store/access your data. More on that at a later date.

To write to file, use `df.to_csv()`

In [23]:
# Let's create a filepath for the full aggregation and write to CSV

output_filepath = os.path.join(data_folder, "aggregated outputs", "block_id_agg_all_people.csv")
block_id_df.to_csv(output_filepath)

In [24]:
# Let's repeat using the 30 to 50 filtered aggregation

output_filepath = os.path.join(data_folder, "aggregated outputs", "block_id_agg_30_to_50.csv")
age_filtered_df.to_csv(output_filepath)